## Capstone Business Case - Battle of the Neighborhoods

Author: Nora Poloni-Gallagher

***

### Description of the problem and discussion of the background


The project aims at clustering the neighbourhoods of Toronto, Canada, based on venue categories. This report will help business owners to identify which neighbourhood is the best place to open a new venue. I will use the k-means clustering in order to cluster the Toronto neighbourhoods, the data will be provided by using Foursquare API.

### Description of the data and how it will be used to solve the problem

The project will rely on the data of Toronto neighbourhoods provided by Foursquare. The types of venues that will be used as categories to cluster the neighbourhoods will also be defined by the data available on Foursquare.

***


### <span style='color:green'> Data Analysis </span>

<span style='background :pink' > Installing BeautifulSoup </span>

In [3]:
! pip install BeautifulSoup4

<span style='background :pink' > Installing Geocoder </span>

In [9]:
! pip install geocoder

<span style='background :pink' > Importing Data </span>

In [5]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for table_cell in soup.find_all('td'):
    try:
        postal_code = table_cell.p.b.text # get the postal code
        postal_code_investigate = table_cell.span.text
        neighborhoods_data = table_cell.span.text # get the rest of the data in the cell
        borough = neighborhoods_data.split('(')[0] # get the borough in the cell
        
        # if the cell is not assigned then ignore it
        if neighborhoods_data == 'Not assigned':
            neighborhoods = []
        # else process the data and add it to the dictionary
        else:
            postal_codes_dict[postal_code] = {}
            try:
                neighborhoods = neighborhoods_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                neighborhoods = neighborhoods.replace('(', ' ')
                neighborhoods = neighborhoods.replace(')', ' ')

                neighborhoods_names = neighborhoods.split('/')
                neighborhoods_clean = ', '.join([name.strip() for name in neighborhoods_names])
            except:
                borough = borough.strip('\n')
                neighborhoods_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_codes_dict[postal_code]['borough'] = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhoods_clean
    except:
        pass
    
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

103

In [10]:
toronto_data.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
import geocoder
from geopy.geocoders import Nominatim
# convert an address into latitude and longitude values
# get latitude and longitude using geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude,longitude )

43.648690000000045 -79.38543999999996


In [15]:
#read geospatial data file
geotable = pd.read_csv('/Users/N/Downloads/Geospatial_Coordinates.csv')
geotable.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


<span style='background :pink' > Combining the data </span>

In [16]:
toronto_data.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [17]:
geotable.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [19]:
combined_data = toronto_data.join(geotable, how='inner')
combined_data

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M9N,43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,M9P,43.696319,-79.532242
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,M9R,43.688905,-79.554724
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M9V,43.739416,-79.588437


In [20]:
combined_data.shape

(103, 6)

<span style='background :pink' > K-means clustering of Toronto neighbourhoods </span>

In [21]:
from geopy.geocoders import Nominatim

In [22]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


In [25]:
import folium

In [26]:
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighborhood in zip(combined_data['Latitude'], combined_data['Longitude'], combined_data['Borough'], combined_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

In [30]:
CLIENT_ID = 'CXNBBR4LMG3B44CRLCQ4LID1W3QTXRUWYMAKIXGGWMY4244X' 
CLIENT_SECRET = 'W3PEMGMDJZWBYN4WUCZEYZNPPRY4Y5IA2Q2X1UGHSZZKI2P5'
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
venues_in_toronto = getNearbyVenues(combined_data['Neighborhood'], combined_data['Latitude'], combined_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [32]:
venues_in_toronto.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy’s,Fast Food Restaurant
1,Victoria Village,43.784535,-79.160497,Great Shine Window Cleaning,Home Service
2,Victoria Village,43.784535,-79.160497,Royal Canadian Legion,Bar
3,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,Bank
4,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,Electronics Store


In [33]:
venues_in_toronto.groupby('Neighbourhood').head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy’s,Fast Food Restaurant
1,Victoria Village,43.784535,-79.160497,Great Shine Window Cleaning,Home Service
2,Victoria Village,43.784535,-79.160497,Royal Canadian Legion,Bar
3,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,Bank
4,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,Electronics Store
...,...,...,...,...,...
1317,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.739416,-79.588437,Sheriff's No Frills,Grocery Store
1321,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054,Economy Rent A Car,Rental Car Location
1322,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054,Logistics Distribution,Bar
1323,"Mimico NW, The Queensway West, South of Bloor,...",43.706748,-79.594054,Saand Rexdale,Drugstore


<span style='background :pink' > Venue categories </span>

In [35]:
venues_in_toronto.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Accessories Store,"Wexford, Maryvale",43.718518,-79.464763,Ardene Shoes Outlet
Adult Boutique,"Willowdale, Newtonbrook",43.665860,-79.383160,Seduction
Airport,"Richmond, Adelaide, King",43.737473,-79.394420,Toronto Downsview Airport (YZD)
Airport Food Court,Forest Hill North & West,43.628947,-79.394420,Billy Bishop Café
Airport Gate,Forest Hill North & West,43.628947,-79.394420,Gate 8
...,...,...,...,...
Warehouse Store,Bayview Village,43.705369,-79.349372,Costco
Wine Bar,"Kingsview Village, St. Phillips, Martin Grove ...",43.653206,-79.400049,Paris Paris Bar
Wings Joint,Enclave of M5E,43.628841,-79.520999,Wingporium


In [36]:
toronto_venue_cat = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")
toronto_venue_cat

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1321,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_venue_cat['Neighbourhood'] = venues_in_toronto['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [toronto_venue_cat.columns[-1]] + list(toronto_venue_cat.columns[:-1])
toronto_venue_cat = toronto_venue_cat[fixed_columns]

toronto_venue_cat.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_grouped = toronto_venue_cat.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0,0.047619
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,...,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


<span style='background :pink' > Top 10 venue categories </span>

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery,Convenience Store,Pet Store,Climbing Gym,Restaurant,Burrito Place
1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Health Food Store,Sushi Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Shopping Mall,Mobile Phone Shop,Deli / Bodega,Restaurant
2,Bayview Village,Sandwich Place,Indian Restaurant,Yoga Studio,Middle Eastern Restaurant,Burger Joint,Restaurant,Pizza Place,Pharmacy,Coffee Shop,Liquor Store
3,"Bedford Park, Lawrence Manor East",Gastropub,Café,Coffee Shop,Farmers Market,New American Restaurant,Diner,Restaurant,Jazz Club,Japanese Restaurant,Italian Restaurant
4,"Birch Cliff, Cliffside West",Coffee Shop,Café,Seafood Restaurant,Gym / Fitness Center,Thai Restaurant,Restaurant,Smoke Shop,Lounge,Speakeasy,Hotel
5,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,American Restaurant,Bakery,Café,Yoga Studio,Gay Bar,Pet Store,Park,Neighborhood,Middle Eastern Restaurant
6,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Gym / Fitness Center,Skate Park,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
7,Caledonia-Fairbanks,Park,Yoga Studio,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
8,Cedarbrae,Gym,Athletics & Sports,Caribbean Restaurant,Japanese Restaurant,Café,Yoga Studio,Discount Store,Dim Sum Restaurant,Diner,Dog Run
9,Central Bay Street,Pharmacy,Pizza Place,Coffee Shop,Supermarket,Discount Store,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run


<span style='background :pink' > Clustering the neighbourhoods </span>

In [43]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [44]:
# set number of clusters
k_num_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

In [45]:
kmeans.labels_[0:100]

array([0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [46]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [47]:
toronto_merged = combined_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')
toronto_merged.head(10)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353,4.0,Fast Food Restaurant,Yoga Studio,Falafel Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497,0.0,Bar,Home Service,Yoga Studio,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711,0.0,Restaurant,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Bank,Electronics Store,Donut Shop,Breakfast Spot,Distribution Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Insurance Office,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,M7A,Queen's Park,Ontario Provincial Government,M1H,43.773136,-79.239476,0.0,Lounge,Bank,Fried Chicken Joint,Bakery,Athletics & Sports,Thai Restaurant,Gas Station,Hakka Restaurant,Caribbean Restaurant,Donut Shop
5,M9A,Etobicoke,Islington Avenue,M1J,43.744734,-79.239476,0.0,Playground,Health & Beauty Service,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
6,M1B,Scarborough,"Malvern, Rouge",M1K,43.727929,-79.262029,0.0,Chinese Restaurant,Coffee Shop,Convenience Store,Department Store,Hobby Shop,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
7,M3B,North York,Don Mills North,M1L,43.711112,-79.284577,0.0,Bus Line,Bakery,Ice Cream Shop,Soccer Field,Park,Intersection,Metro Station,Bus Station,Diner,Yoga Studio
8,M4B,East York,"Parkview Hill, Woodbine Gardens",M1M,43.716316,-79.239476,0.0,Motel,Skating Rink,American Restaurant,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848,0.0,College Stadium,Café,Skating Rink,General Entertainment,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [48]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

In [49]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [50]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

In [51]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,-79.160497,0.0,Bar,Home Service,Yoga Studio,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
2,Downtown Toronto,-79.188711,0.0,Restaurant,Medical Center,Rental Car Location,Intersection,Mexican Restaurant,Bank,Electronics Store,Donut Shop,Breakfast Spot,Distribution Center
3,North York,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Insurance Office,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
4,Queen's Park,-79.239476,0.0,Lounge,Bank,Fried Chicken Joint,Bakery,Athletics & Sports,Thai Restaurant,Gas Station,Hakka Restaurant,Caribbean Restaurant,Donut Shop
5,Etobicoke,-79.239476,0.0,Playground,Health & Beauty Service,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,-79.532242,0.0,Baseball Field,Food Service,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
99,Downtown Toronto,-79.532242,0.0,Discount Store,Intersection,Pizza Place,Sandwich Place,Coffee Shop,Middle Eastern Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dessert Shop,Yoga Studio
100,East TorontoBusiness reply mail Processing Cen...,-79.554724,0.0,Park,Pizza Place,Bus Line,Sandwich Place,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
101,Etobicoke,-79.588437,0.0,Grocery Store,Pharmacy,Pizza Place,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Sandwich Place,Diner,Dessert Shop,Dim Sum Restaurant


### We have successfully identified the 10 most common venue types in each Toronto neighbourhood.